# 22.1. Параметризация тестов в pytest
**Параметризация** — это способ запустить один и тот же тест с разным набором входных параметров.

Для организации параметризованных тестов существует два способа:
- Через значение параметра params фикстуры, в который нужно передать массив значений. То есть фактически фикстура в данном случае представляет собой обёртку, передающую параметры. А в сам тест они передаются через атрибут param объекта request.
- Через декоратор @pytest.mark.parametrize, в который передаётся список названий переменных и массив их значений.
## Параметризация с помощью фикстуры

In [1]:
import pytest

def python_string_slicer(str):
   if len(str) < 50 or "python" in str:
       return str
   else:
       return str[0:50]


@pytest.fixture(scope="function", params=[
   ("Короткая строка", "Короткая строка"),
   ("Длинная строка, не то чтобы прям очень длинная, но достаточно для нашего теста, и в ней нет названия языка"
    , "Длинная строка, не то чтобы прям очень длинная, но"),
   ("Короткая строка со словом python", "Короткая строка со словом python"),
   ("Длинная строка, нам достаточно будет для проверки, и в ней есть слово python"
    , "Длинная строка, нам достаточно будет для проверки, и в ней есть слово python")
])
def param_fun(request):
   return request.param


def test_python_string_slicer(param_fun):
   (input, expected_output) = param_fun
   result = python_string_slicer(input)
   print ("Входная строка: {0}\nВыходная строка: {1}\nОжидаемое значение: {2}".format(input, result, expected_output))
   assert result == expected_output

Если мы передаём id тестов прямо в фикстуру, то наша фикстура превратится вот в такой код:

In [ ]:
@pytest.fixture(scope="function", params=[
   ("Короткая строка", "Короткая строка"),
   ("Длинная строка, не то чтобы прям очень длинная, но достаточно для нашего теста, и в ней нет названия языка"
    , "Длинная строка, не то чтобы прям очень длинная, но"),
   ("Короткая строка со словом python", "Короткая строка со словом python"),
   ("Длинная строка, нам достаточно будет для проверки, и в ней есть слово python"
    , "Длинная строка, нам достаточно будет для проверки, и в ней есть слово python"),
], ids=["len < 50", "len > 50", "len < 50 contains python", "len > 50 contains python"])
def param_fun(request):
   return request.param

вынесем генерацию названий в отдельную функцию:

In [ ]:
def generate_id(val):
   return "params: {0}".format(str(val))


@pytest.fixture(scope="function", params=[
   ("Короткая строка", "Короткая строка"),
   ("Длинная строка, не то чтобы прям очень длинная, но достаточно для нашего теста, и в ней нет названия языка"
    , "Длинная строка, не то чтобы прям очень длинная, но"),
   ("Короткая строка со словом python", "Короткая строка со словом python"),
   ("Длинная строка, нам достаточно будет для проверки, и в ней есть слово python"
    , "Длинная строка, нам достаточно будет для проверки, и в ней есть слово python"),
], ids=generate_id)
def param_fun_generated(request):
   return request.param


def test_python_string_slicer_generated(param_fun_generated):
   (input, expected_output) = param_fun_generated
   result = python_string_slicer(input)
   print("Входная строка: {0}\nВыходная строка: {1}\nОжидаемое значение: {2}".format(input, result, expected_output))
   assert result == expected_output

Плюсы метода:
- параметры отдельно от тестов (удобно хранить их для нескольких тестов)
- код тестов сокращается
Минусы:
- для каждого параметра своя функция
- неудобно, если придется переименовать функцию
- сложно понять к какому тесту относится параметр (нужно увидеть куда передается аргумент функции с параметрами)

## Параметризация с помощью pytest.mark.parametrize
позволяет более гибко управлять набором тестов, создаваемых с помощью параметризации. Если указать несколько меток с разными параметрами, то тест будет запущен со всеми возможными наборами параметров (то есть мы имеем декартово произведение). Напишем простой тест, чтобы это продемонстрировать:

In [ ]:
import pytest

@pytest.mark.parametrize("x", [1, 2, 3])
@pytest.mark.parametrize("y", [10, 11])
def test_multiply_params(x, y):
   print("x: {0}, y: {1}".format(x, y))
   assert True

В фикстуру метке parametrize также можно передать параметр ids, и он точно так же будет ответственен за отображение параметров в выводе. Точно так же нам доступны два варианта передачи ids в фикстуру — в виде имени функции или набора значений. Для начала давайте посмотрим, как передаётся набор значений:

In [ ]:
@pytest.mark.parametrize("x", [-1, 0, 1], ids=["negative", "zero", "positive"])
@pytest.mark.parametrize("y", [100, 1000], ids=["3 digit", "4 digit"])
def test_multiply_params(x, y):
   print("x: {0}, y: {1}".format(x, y))
   assert True

Проделаем аналогичные операции с генератором имён параметров:

In [ ]:
def ids_x(val):
   return "x=({0})".format(str(val))


def ids_y(val):
   return "y=({0})".format(str(val))


@pytest.mark.parametrize("x", [-1, 0, 1], ids=ids_x)
@pytest.mark.parametrize("y", [100, 1000], ids=ids_y)
def test_multiply_params(x, y):
   print("x: {0}, y: {1}".format(x, y))
   assert True

Памятка по способам параметризации тестов:
1. @pytest.fixture(scope,values,ids)
   - scope — область действия;
   - values — значения, которые будут использованы в тестах;
   - ids — уникальный идентификатор значения параметра.
+ Параметры стоят отдельно от тестов, и это позволяет хранить их в одном месте тестового класса.
– Для каждого параметра придётся писать свою функцию.
– Сложно понять, к какому тесту относится данный параметр, придётся обратиться к коду тестов.

2. @pytest.mark.parametrize(param, values, ids)
   - param — имя параметра, использующееся в тестовой функции;
   - values — значения, которые будут использованы в тестах;
   - ids — уникальный идентификатор значения параметра.
+ Используется вместе с тестом, т.е. мы легко получаем доступ к возможным значениям параметров.
+ Не требует написания дополнительного кода.

– При изменении имени параметра в аргументах тестовой функции придётся поменять его и в соответствующей фикстуре.